# Подготовка данных. Создание фичей.

В проекте используются заранее загруженные данные с MOEX, помещенные в БД.  
Для загрузки используется https://github.com/Alkor135/quote_download

In [172]:
import sqlite3
from pathlib import Path
import numpy as np

import pandas as pd

In [173]:
# Настройки для отображения широкого df pandas
pd.options.display.width = 1200
pd.options.display.max_colwidth = 100
pd.options.display.max_columns = 100

Загрузка имен таблиц из БД.

In [174]:
# Укажите путь к вашей базе данных SQLite
db_path = Path(r'C:\Users\Alkor\gd\data_quote_db\RTS_futures_options_day.db')

# Установите соединение с базой данных
conn = sqlite3.connect(db_path)

# Получение df с именами таблиц
query = "SELECT name FROM sqlite_master WHERE type='table'"
df_table = pd.read_sql_query(query, conn)
# table = pd.read_sql_query(query, conn).iloc[0, 0]
df_table

,name
0,Futures
1,Options
2,sqlite_sequence


Загрузка данных по фьючерсам.

In [175]:
# Выполните SQL-запрос и загрузите результаты в DataFrame
query = f"SELECT * FROM {df_table.iloc[0, 0]}"
df_fut = pd.read_sql_query(query, conn)
# Проверьте данные
print(df_fut.columns.tolist())
df_fut

['TRADEDATE', 'SECID', 'OPEN', 'LOW', 'HIGH', 'CLOSE', 'VOLUME', 'OPENPOSITION', 'SHORTNAME', 'LSTTRADE']


,TRADEDATE,SECID,OPEN,LOW,HIGH,CLOSE,VOLUME,OPENPOSITION,SHORTNAME,LSTTRADE
0,2015-01-05,RIH5_2015,76930.0,72470.0,78980.0,74600.0,372848,751996,RTS-3.15,2015-03-16
1,2015-01-06,RIH5_2015,74470.0,71200.0,74610.0,73480.0,319307,731236,RTS-3.15,2015-03-16
2,2015-01-08,RIH5_2015,73490.0,71000.0,81380.0,79980.0,537469,751010,RTS-3.15,2015-03-16
3,2015-01-09,RIH5_2015,79950.0,74450.0,81050.0,77650.0,592715,865312,RTS-3.15,2015-03-16
4,2015-01-12,RIH5_2015,77210.0,73180.0,77550.0,73900.0,440908,897314,RTS-3.15,2015-03-16
...,...,...,...,...,...,...,...,...,...,...
2537,2025-02-12,RIH5,98600.0,98120.0,101480.0,100700.0,124069,115388,RTS-3.25,2025-03-20
2538,2025-02-13,RIH5,100570.0,100080.0,111010.0,109930.0,247083,107578,RTS-3.25,2025-03-20
2539,2025-02-14,RIH5,109750.0,106470.0,114130.0,108620.0,282242,107550,RTS-3.25,2025-03-20
2540,2025-02-17,RIH5,108960.0,108050.0,115370.0,115170.0,183356,123784,RTS-3.25,2025-03-20


In [176]:
# Преобразуем в datetime
df_fut['TRADEDATE'] = pd.to_datetime(df_fut['TRADEDATE'])
df_fut['LSTTRADE'] = pd.to_datetime(df_fut['LSTTRADE'])

# Вычисляем разницу в днях
df_fut['date_diff'] = (df_fut['LSTTRADE'] - df_fut['TRADEDATE']).dt.days

# Нормализация в диапазон [-1, 1]
min_diff = df_fut['date_diff'].min()
max_diff = df_fut['date_diff'].max()

df_fut['norm_date_diff'] = 2 * (df_fut['date_diff'] - min_diff) / (max_diff - min_diff) - 1

df_fut = df_fut.drop(columns=['date_diff'])

df_fut

,TRADEDATE,SECID,OPEN,LOW,HIGH,CLOSE,VOLUME,OPENPOSITION,SHORTNAME,LSTTRADE,norm_date_diff
0,2015-01-05,RIH5_2015,76930.0,72470.0,78980.0,74600.0,372848,751996,RTS-3.15,2015-03-16,0.352941
1,2015-01-06,RIH5_2015,74470.0,71200.0,74610.0,73480.0,319307,731236,RTS-3.15,2015-03-16,0.333333
2,2015-01-08,RIH5_2015,73490.0,71000.0,81380.0,79980.0,537469,751010,RTS-3.15,2015-03-16,0.294118
3,2015-01-09,RIH5_2015,79950.0,74450.0,81050.0,77650.0,592715,865312,RTS-3.15,2015-03-16,0.274510
4,2015-01-12,RIH5_2015,77210.0,73180.0,77550.0,73900.0,440908,897314,RTS-3.15,2015-03-16,0.215686
...,...,...,...,...,...,...,...,...,...,...,...
2537,2025-02-12,RIH5,98600.0,98120.0,101480.0,100700.0,124069,115388,RTS-3.25,2025-03-20,-0.313725
2538,2025-02-13,RIH5,100570.0,100080.0,111010.0,109930.0,247083,107578,RTS-3.25,2025-03-20,-0.333333
2539,2025-02-14,RIH5,109750.0,106470.0,114130.0,108620.0,282242,107550,RTS-3.25,2025-03-20,-0.352941
2540,2025-02-17,RIH5,108960.0,108050.0,115370.0,115170.0,183356,123784,RTS-3.25,2025-03-20,-0.411765


Загрузка данных по опционам.

In [177]:
# Выполните SQL-запрос и загрузите результаты в DataFrame
query = f"SELECT * FROM {df_table.iloc[1, 0]}"
df_opt = pd.read_sql_query(query, conn)
# Закрываем соединение
conn.close()

print(df_opt.columns.tolist())
df_opt

['ID', 'TRADEDATE', 'SECID', 'OPENPOSITION', 'NAME', 'LSTTRADE', 'OPTIONTYPE', 'STRIKE']


,ID,TRADEDATE,SECID,OPENPOSITION,NAME,LSTTRADE,OPTIONTYPE,STRIKE
0,1,2015-01-05,RI100000BA5_2015,18004,RTS-3.15,2015-01-15,C,100000
1,2,2015-01-05,RI100000BB5_2015,246,RTS-3.15,2015-02-16,C,100000
2,3,2015-01-05,RI100000BC5_2015,8784,RTS-3.15,2015-03-16,C,100000
3,4,2015-01-05,RI100000BM5_2015,3352,RTS-3.15,2015-01-15,P,100000
4,5,2015-01-05,RI100000BN5_2015,462,RTS-3.15,2015-02-16,P,100000
...,...,...,...,...,...,...,...,...
817161,817498,2025-02-18,RI97500BC5A,0,RTS-3.25,2025-03-06,C,97500
817162,817499,2025-02-18,RI97500BN5,4190,RTS-3.25,2025-02-20,P,97500
817163,817500,2025-02-18,RI97500BN5D,204,RTS-3.25,2025-02-27,P,97500
817164,817501,2025-02-18,RI97500BO5,312,RTS-3.25,2025-03-20,P,97500


Далее добавляем в дата фрейм с фьючерсами данные по отрытому интересу опционов колл и пут.

In [178]:
# Группируем по дате и типу опциона и суммируем 'OPENPOSITION'
df_oi_opt = df_opt.groupby(['TRADEDATE', 'OPTIONTYPE'])['OPENPOSITION'].sum().unstack()

# Преобразуем даты в datetime64
df_opt['TRADEDATE'] = pd.to_datetime(df_opt['TRADEDATE'])
df_fut['TRADEDATE'] = pd.to_datetime(df_fut['TRADEDATE'])
df_oi_opt.index = pd.to_datetime(df_oi_opt.index)  # Приведение индекса к datetime64

# Объединяем датафреймы по полю 'TRADEDATE'
df = pd.merge(df_fut, df_oi_opt, on='TRADEDATE', how='inner')

# Переименовываем колонки
df.rename(columns={'OPENPOSITION': 'OI', 'C': 'OI_C', 'P': 'OI_P'}, inplace=True)

# Удаляем ненужные колонки
df = df.drop(columns=['SECID', 'SHORTNAME', 'LSTTRADE'])

df

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME,OI,norm_date_diff,OI_C,OI_P
0,2015-01-05,76930.0,72470.0,78980.0,74600.0,372848,751996,0.352941,261428,159204
1,2015-01-06,74470.0,71200.0,74610.0,73480.0,319307,731236,0.333333,265924,157238
2,2015-01-08,73490.0,71000.0,81380.0,79980.0,537469,751010,0.294118,275540,164374
3,2015-01-09,79950.0,74450.0,81050.0,77650.0,592715,865312,0.274510,285592,167670
4,2015-01-12,77210.0,73180.0,77550.0,73900.0,440908,897314,0.215686,293664,175758
...,...,...,...,...,...,...,...,...,...,...
2537,2025-02-12,98600.0,98120.0,101480.0,100700.0,124069,115388,-0.313725,45156,81848
2538,2025-02-13,100570.0,100080.0,111010.0,109930.0,247083,107578,-0.333333,30622,58224
2539,2025-02-14,109750.0,106470.0,114130.0,108620.0,282242,107550,-0.352941,53530,66822
2540,2025-02-17,108960.0,108050.0,115370.0,115170.0,183356,123784,-0.411765,54902,73364


In [179]:
# # Группируем по дате и типу опциона и суммируем 'OPENPOSITION'
# df_oi_opt = df_opt.groupby(['TRADEDATE', 'OPTIONTYPE'])['OPENPOSITION'].sum().unstack()

# df_opt['TRADEDATE'] = pd.to_datetime(df_opt['TRADEDATE'])

# # Объединяем датафреймы по полю 'TRADEDATE'
# df = pd.merge(df_fut, df_oi_opt, on='TRADEDATE', how='inner')
# df.rename(columns={'OPENPOSITION': 'OI', 'C': 'OI_C', 'P': 'OI_P'}, inplace=True)
# # Удаляем ненужные колонки, например, 'OPEN' и 'VOLUME'
# df = df.drop(columns=['SECID', 'SHORTNAME', 'LSTTRADE'])
# df

Создание фичей из объема. Объемы каждый день новые(начинаются с 0). Нормализация к среднему значению за 10 предыдущих баров. Диапазон от -1 до 1.

In [180]:
# Вычисляем скользящее среднее за 10 предыдущих значений, включая текущее
df['MA_10'] = df['VOLUME'].shift(1).rolling(window=10, min_periods=1).mean()
# Нормализуем колонку 'VOLUME' к среднему значению от -1 до 1
df['norm_vol'] = 2 * (df['VOLUME'] - df['MA_10']) / (df['MA_10'] + 1)
df = df.drop(columns=['MA_10'])
df

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME,OI,norm_date_diff,OI_C,OI_P,norm_vol
0,2015-01-05,76930.0,72470.0,78980.0,74600.0,372848,751996,0.352941,261428,159204,NaN
1,2015-01-06,74470.0,71200.0,74610.0,73480.0,319307,731236,0.333333,265924,157238,-0.287199
2,2015-01-08,73490.0,71000.0,81380.0,79980.0,537469,751010,0.294118,275540,164374,1.106058
3,2015-01-09,79950.0,74450.0,81050.0,77650.0,592715,865312,0.274510,285592,167670,0.892175
4,2015-01-12,77210.0,73180.0,77550.0,73900.0,440908,897314,0.215686,293664,175758,-0.064430
...,...,...,...,...,...,...,...,...,...,...,...
2537,2025-02-12,98600.0,98120.0,101480.0,100700.0,124069,115388,-0.313725,45156,81848,0.540250
2538,2025-02-13,100570.0,100080.0,111010.0,109930.0,247083,107578,-0.333333,30622,58224,2.850324
2539,2025-02-14,109750.0,106470.0,114130.0,108620.0,282242,107550,-0.352941,53530,66822,2.803204
2540,2025-02-17,108960.0,108050.0,115370.0,115170.0,183356,123784,-0.411765,54902,73364,0.721611


Создание фичей на основе изменения открытого интереса к предыдущему значению.

In [181]:
# Вычисляем изменение значения 'OI' по сравнению с предыдущим значением
df['OI_diff'] = df['OI'].diff()

# Нормализуем изменения, разделив разницу на предыдущее значение 'OI'
df['norm_oi_prev'] = df['OI_diff'] / df['OI'].shift(1)

df = df.drop(columns=['OI_diff'])
df

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME,OI,norm_date_diff,OI_C,OI_P,norm_vol,norm_oi_prev
0,2015-01-05,76930.0,72470.0,78980.0,74600.0,372848,751996,0.352941,261428,159204,NaN,NaN
1,2015-01-06,74470.0,71200.0,74610.0,73480.0,319307,731236,0.333333,265924,157238,-0.287199,-0.027607
2,2015-01-08,73490.0,71000.0,81380.0,79980.0,537469,751010,0.294118,275540,164374,1.106058,0.027042
3,2015-01-09,79950.0,74450.0,81050.0,77650.0,592715,865312,0.274510,285592,167670,0.892175,0.152198
4,2015-01-12,77210.0,73180.0,77550.0,73900.0,440908,897314,0.215686,293664,175758,-0.064430,0.036983
...,...,...,...,...,...,...,...,...,...,...,...,...
2537,2025-02-12,98600.0,98120.0,101480.0,100700.0,124069,115388,-0.313725,45156,81848,0.540250,0.111938
2538,2025-02-13,100570.0,100080.0,111010.0,109930.0,247083,107578,-0.333333,30622,58224,2.850324,-0.067685
2539,2025-02-14,109750.0,106470.0,114130.0,108620.0,282242,107550,-0.352941,53530,66822,2.803204,-0.000260
2540,2025-02-17,108960.0,108050.0,115370.0,115170.0,183356,123784,-0.411765,54902,73364,0.721611,0.150944


In [182]:
# Фичи 'OI'
# df['norm_oi_c/oi'] = df['OI_C'] / df['OI']
df['norm_oi_c/oi'] = (df['OI'] - df['OI_C']) / df['OI'] * 2 - 1
df['norm_oi_p/oi'] = (df['OI'] - df['OI_P']) / df['OI'] * 2 - 1
df['norm_oi_c/oi_p'] = (df['OI_C'] - df['OI_P']) / df['OI_P']

df

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME,OI,norm_date_diff,OI_C,OI_P,norm_vol,norm_oi_prev,norm_oi_c/oi,norm_oi_p/oi,norm_oi_c/oi_p
0,2015-01-05,76930.0,72470.0,78980.0,74600.0,372848,751996,0.352941,261428,159204,NaN,NaN,0.304709,0.576583,0.642094
1,2015-01-06,74470.0,71200.0,74610.0,73480.0,319307,731236,0.333333,265924,157238,-0.287199,-0.027607,0.272673,0.569939,0.691220
2,2015-01-08,73490.0,71000.0,81380.0,79980.0,537469,751010,0.294118,275540,164374,1.106058,0.027042,0.266215,0.562259,0.676299
3,2015-01-09,79950.0,74450.0,81050.0,77650.0,592715,865312,0.274510,285592,167670,0.892175,0.152198,0.339910,0.612463,0.703298
4,2015-01-12,77210.0,73180.0,77550.0,73900.0,440908,897314,0.215686,293664,175758,-0.064430,0.036983,0.345460,0.608258,0.670843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,2025-02-12,98600.0,98120.0,101480.0,100700.0,124069,115388,-0.313725,45156,81848,0.540250,0.111938,0.217319,-0.418657,-0.448294
2538,2025-02-13,100570.0,100080.0,111010.0,109930.0,247083,107578,-0.333333,30622,58224,2.850324,-0.067685,0.430701,-0.082452,-0.474066
2539,2025-02-14,109750.0,106470.0,114130.0,108620.0,282242,107550,-0.352941,53530,66822,2.803204,-0.000260,0.004556,-0.242622,-0.198917
2540,2025-02-17,108960.0,108050.0,115370.0,115170.0,183356,123784,-0.411765,54902,73364,0.721611,0.150944,0.112939,-0.185355,-0.251649


In [183]:
# Добавляем колонку с изменением относительно CLOSE
df['h/c'] = (df['HIGH'] - df['CLOSE']) / df['CLOSE']
df['o/c'] = (df['OPEN'] - df['CLOSE']) / df['CLOSE']
df['l/c'] = (df['LOW'] - df['CLOSE']) / df['CLOSE']
df

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME,OI,norm_date_diff,OI_C,OI_P,norm_vol,norm_oi_prev,norm_oi_c/oi,norm_oi_p/oi,norm_oi_c/oi_p,h/c,o/c,l/c
0,2015-01-05,76930.0,72470.0,78980.0,74600.0,372848,751996,0.352941,261428,159204,NaN,NaN,0.304709,0.576583,0.642094,0.058713,0.031233,-0.028552
1,2015-01-06,74470.0,71200.0,74610.0,73480.0,319307,731236,0.333333,265924,157238,-0.287199,-0.027607,0.272673,0.569939,0.691220,0.015378,0.013473,-0.031029
2,2015-01-08,73490.0,71000.0,81380.0,79980.0,537469,751010,0.294118,275540,164374,1.106058,0.027042,0.266215,0.562259,0.676299,0.017504,-0.081145,-0.112278
3,2015-01-09,79950.0,74450.0,81050.0,77650.0,592715,865312,0.274510,285592,167670,0.892175,0.152198,0.339910,0.612463,0.703298,0.043786,0.029620,-0.041211
4,2015-01-12,77210.0,73180.0,77550.0,73900.0,440908,897314,0.215686,293664,175758,-0.064430,0.036983,0.345460,0.608258,0.670843,0.049391,0.044790,-0.009743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,2025-02-12,98600.0,98120.0,101480.0,100700.0,124069,115388,-0.313725,45156,81848,0.540250,0.111938,0.217319,-0.418657,-0.448294,0.007746,-0.020854,-0.025621
2538,2025-02-13,100570.0,100080.0,111010.0,109930.0,247083,107578,-0.333333,30622,58224,2.850324,-0.067685,0.430701,-0.082452,-0.474066,0.009824,-0.085145,-0.089602
2539,2025-02-14,109750.0,106470.0,114130.0,108620.0,282242,107550,-0.352941,53530,66822,2.803204,-0.000260,0.004556,-0.242622,-0.198917,0.050727,0.010403,-0.019794
2540,2025-02-17,108960.0,108050.0,115370.0,115170.0,183356,123784,-0.411765,54902,73364,0.721611,0.150944,0.112939,-0.185355,-0.251649,0.001737,-0.053920,-0.061822


In [184]:
for i in range(1, 10):
    df[f'c_{i}/c'] = (df['CLOSE'].shift(i) - df['CLOSE']) / df['CLOSE']
    df[f'h_{i}/c'] = (df['HIGH'].shift(i) - df['CLOSE']) / df['CLOSE']
    df[f'o_{i}/c'] = (df['OPEN'].shift(i) - df['CLOSE']) / df['CLOSE']
    df[f'l_{i}/c'] = (df['LOW'].shift(i) - df['CLOSE']) / df['CLOSE']

df

,TRADEDATE,OPEN,LOW,HIGH,CLOSE,VOLUME,OI,norm_date_diff,OI_C,OI_P,norm_vol,norm_oi_prev,norm_oi_c/oi,norm_oi_p/oi,norm_oi_c/oi_p,h/c,o/c,l/c,c_1/c,h_1/c,o_1/c,l_1/c,c_2/c,h_2/c,o_2/c,l_2/c,c_3/c,h_3/c,o_3/c,l_3/c,c_4/c,h_4/c,o_4/c,l_4/c,c_5/c,h_5/c,o_5/c,l_5/c,c_6/c,h_6/c,o_6/c,l_6/c,c_7/c,h_7/c,o_7/c,l_7/c,c_8/c,h_8/c,o_8/c,l_8/c,c_9/c,h_9/c,o_9/c,l_9/c
0,2015-01-05,76930.0,72470.0,78980.0,74600.0,372848,751996,0.352941,261428,159204,NaN,NaN,0.304709,0.576583,0.642094,0.058713,0.031233,-0.028552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-06,74470.0,71200.0,74610.0,73480.0,319307,731236,0.333333,265924,157238,-0.287199,-0.027607,0.272673,0.569939,0.691220,0.015378,0.013473,-0.031029,0.015242,0.074850,0.046952,-0.013745,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-08,73490.0,71000.0,81380.0,79980.0,537469,751010,0.294118,275540,164374,1.106058,0.027042,0.266215,0.562259,0.676299,0.017504,-0.081145,-0.112278,-0.081270,-0.067142,-0.068892,-0.109777,-0.067267,-0.012503,-0.038135,-0.093898,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-09,79950.0,74450.0,81050.0,77650.0,592715,865312,0.274510,285592,167670,0.892175,0.152198,0.339910,0.612463,0.703298,0.043786,0.029620,-0.041211,0.030006,0.048036,-0.053574,-0.085641,-0.053703,-0.039150,-0.040953,-0.083065,-0.039279,0.017128,-0.009272,-0.066710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-12,77210.0,73180.0,77550.0,73900.0,440908,897314,0.215686,293664,175758,-0.064430,0.036983,0.345460,0.608258,0.670843,0.049391,0.044790,-0.009743,0.050744,0.096752,0.081867,0.007442,0.082273,0.101218,-0.005548,-0.039242,-0.005683,0.009608,0.007713,-0.036536,0.009472,0.068742,0.041001,-0.019350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,2025-02-12,98600.0,98120.0,101480.0,100700.0,124069,115388,-0.313725,45156,81848,0.540250,0.111938,0.217319,-0.418657,-0.448294,0.007746,-0.020854,-0.025621,-0.020953,-0.018868,-0.022145,-0.037339,-0.022443,-0.020655,-0.048759,-0.050050,-0.048262,-0.035055,-0.046475,-0.051539,-0.045482,-0.026812,-0.040715,-0.047666,-0.041410,-0.039821,-0.066336,-0.077557,-0.065938,-0.057299,-0.067627,-0.071797,-0.068222,-0.052632,-0.061072,-0.078153,-0.060377,-0.039126,-0.047865,-0.065144,-0.047071,-0.046077,-0.061867,-0.067428
2538,2025-02-13,100570.0,100080.0,111010.0,109930.0,247083,107578,-0.333333,30622,58224,2.850324,-0.067685,0.430701,-0.082452,-0.474066,0.009824,-0.085145,-0.089602,-0.083963,-0.076867,-0.103066,-0.107432,-0.103157,-0.101246,-0.104248,-0.118166,-0.104521,-0.102884,-0.128627,-0.129810,-0.128172,-0.116074,-0.126535,-0.131174,-0.125625,-0.108524,-0.121259,-0.127627,-0.121896,-0.120440,-0.144728,-0.155008,-0.144365,-0.136450,-0.145911,-0.149732,-0.146457,-0.132175,-0.139907,-0.155554,-0.139270,-0.119804,-0.127809,-0.143637
2539,2025-02-14,109750.0,106470.0,114130.0,108620.0,282242,107550,-0.352941,53530,66822,2.803204,-0.000260,0.004556,-0.242622,-0.198917,0.050727,0.010403,-0.019794,0.012060,0.022003,-0.074112,-0.078623,-0.072915,-0.065734,-0.092248,-0.096667,-0.092340,-0.090407,-0.093445,-0.107531,-0.093721,-0.092064,-0.118118,-0.119315,-0.117658,-0.105413,-0.116001,-0.120696,-0.115080,-0.097772,-0.110661,-0.117106,-0.111305,-0.109832,-0.134414,-0.144817,-0.134045,-0.126036,-0.135610,-0.139477,-0.136163,-0.121709,-0.129534,-0.145369
2540,2025-02-17,108960.0,108050.0,115370.0,115170.0,183356,123784,-0.411765,54902,73364,0.721611,0.150944,0.112939,-0.185355,-0.251649,0.00173